In [152]:
%load_ext autoreload
%autoreload 2

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re
from tqdm import tqdm
import glob
from utils import *
import pathlib
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import requests

store_list    = ["ricardoeletro", "magazineluiza"]

dataset_pages = f"data/heuristic_pages/*/*"
dataset_links = f"data/heusristic_links/heusristic_links/*"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
dataset_data = [(url, pathlib.Path(url).stem.split('_')[0] ) for url in glob.glob(dataset_links) if len(url.split('_'))>3]

In [5]:
datasets = []

dataset_idx = 0
data_raw_s = pd.read_csv(dataset_data[dataset_idx][0])
data_raw_s = data_raw_s.dropna().drop_duplicates()
data_raw_s['store'] = np.array([dataset_data[dataset_idx][1]] * data_raw_s.shape[0])

for dataset_idx in range(1, len(dataset_data)):
    data_raw = pd.read_csv(dataset_data[dataset_idx][0])
    data_raw = data_raw.dropna().drop_duplicates()
    data_raw['store'] = np.array([dataset_data[dataset_idx][1]] * data_raw.shape[0])
    data_raw_s = pd.merge(data_raw_s, data_raw, how='outer')
    #datasets.append(data_raw)

In [58]:
x_pages = [f'data/heuristic_pages/{store}/page_{p_id}.html' for (store, p_id) in zip(data_raw_s['store'], data_raw_s['id'])]
y = data_raw_s['label'].values
u_class = data_raw_s['store']
l = data_raw_s['links'].values

In [22]:
X, idx_arr = pages_to_matrix(x_pages)
y = y[idx_arr]
u_class = u_class[idx_arr]
l = l[idx_arr]
X.shape, y.shape, u_class.shape, l.shape


Invalid Links 4


In [63]:
df = pd.DataFrame()
df['store'] = u_class
df['link'] = l
df['text'] = X
df['label'] = y
df.to_csv("text_store_label.csv", index=False)

### Test

In [136]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process.kernels import RBF
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

In [113]:
clss = [GaussianNB(), MultinomialNB(), KNeighborsClassifier(), SVC(gamma='auto'), 
        RandomForestClassifier(n_estimators=200), GradientBoostingClassifier(n_estimators=200)]

# T

In [114]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)


In [142]:
CV = CountVectorizer()
    
for cls in clss:

    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        #('tfidf', TfidfTransformer()),
        ('tranf', DenseTransformer()),
        ('clf', cls),
    ])
     
    text_clf.fit(X_train, y_train)  
    predicted = text_clf.predict(X_test)
        
    print(cls.__class__.__name__, np.mean(predicted == y_test))

GaussianNB 0.767097966728281
MultinomialNB 0.9537892791127541
KNeighborsClassifier 0.9759704251386322
SVC 0.9593345656192237
RandomForestClassifier 0.9796672828096118
GradientBoostingClassifier 0.9796672828096118


In [143]:
class DenseTransformer(MinMaxScaler):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [144]:
CV = CountVectorizer()
    
for cls in clss:
    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('tranf', DenseTransformer()),
        ('clf', cls),
    ])
    
    
    text_clf.fit(X_train, y_train)  
    predicted = text_clf.predict(X_test)
        
    print(cls.__class__.__name__, np.mean(predicted == y_test))

GaussianNB 0.7689463955637708
MultinomialNB 0.9630314232902033
KNeighborsClassifier 0.9685767097966729
SVC 0.7652495378927912
RandomForestClassifier 0.9815157116451017
GradientBoostingClassifier 0.9796672828096118


# Leave One Out

In [147]:
from sklearn.model_selection import LeaveOneGroupOut

In [162]:
acc_results = []
header = [cls.__class__.__name__ for cls in clss]
for train_index, test_index in tqdm(logo.split(X, y, u_class)):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    stores_results = []
    
    for cls in clss:
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            ('tfidf', TfidfTransformer()),
            ('tranf', DenseTransformer()),
            ('clf', cls),
        ])


        text_clf.fit(X_train, y_train)  
        predicted = text_clf.predict(X_test)
        acc = np.mean(predicted == y_test)
        stores_results.append(acc)
    acc_results.append(stores_results)
acc_results = pd.DataFrame(acc_results, columns=header)
acc_results.describe()

,GaussianNB,MultinomialNB,KNeighborsClassifier,SVC,RandomForestClassifier,GradientBoostingClassifier
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.635668,0.823733,0.761272,0.748537,0.836681,0.827880
std,0.286603,0.176425,0.132788,0.145035,0.190434,0.241726
min,0.100503,0.425366,0.561111,0.516667,0.420488,0.425366
25%,0.464789,0.727520,0.640187,0.636098,0.723077,0.718310
50%,0.727520,0.902821,0.753846,0.723077,0.910082,0.953271
75%,0.821317,0.929907,0.874659,0.866485,0.993730,0.994444
max,0.933333,0.969231,0.931034,0.924765,0.996226,1.000000


In [164]:
acc_results = []
header = [cls.__class__.__name__ for cls in clss]
for train_index, test_index in tqdm(logo.split(X, y, u_class)):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    stores_results = []
    
    for cls in clss:
        text_clf = Pipeline([
            ('vect', CountVectorizer()),
            #('tfidf', TfidfTransformer()),
            ('tranf', DenseTransformer()),
            ('clf', cls),
        ])


        text_clf.fit(X_train, y_train)  
        predicted = text_clf.predict(X_test)
        acc = np.mean(predicted == y_test)
        stores_results.append(acc)
    acc_results.append(stores_results)
acc_results = pd.DataFrame(acc_results, columns=header)
acc_results.describe()

,GaussianNB,MultinomialNB,KNeighborsClassifier,SVC,RandomForestClassifier,GradientBoostingClassifier
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.598109,0.822047,0.774460,0.786776,0.866772,0.869259
std,0.290799,0.195329,0.188376,0.185893,0.202482,0.206566
min,0.110553,0.425366,0.421463,0.419512,0.425366,0.425366
25%,0.415888,0.690141,0.723077,0.718310,0.723077,0.718310
50%,0.586341,0.924528,0.847411,0.803738,0.976636,0.994444
75%,0.803774,0.969231,0.859296,0.949749,0.996226,0.996226
max,0.977778,1.000000,1.000000,0.996226,1.000000,1.000000
